In [1]:
# importing libraries
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import numpy as np
import pickle
import random

In [2]:
# prepared input dataframe
df = pd.read_csv('pima_dataset.csv')
df = df.drop(['Insulin'], axis=1)
df = df.drop(['DiabetesPedigreeFunction'], axis=1)

In [3]:
#prep X, y
y = df['Outcome']
X = df.drop(['Outcome'], axis=1)
print('X Shape: ' + str(X.shape))
print('y Shape: ' + str(y.shape))

X Shape: (768, 6)
y Shape: (768,)


In [4]:
#train, test split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=random.seed(69))
print('X_train Shape: ' + str(X_train.shape))
print('X_test Shape: ' + str(X_test.shape))
print('y_train Shape: ' + str(y_train.shape))
print('y_test Shape: ' + str(y_test.shape))

X_train Shape: (614, 6)
X_test Shape: (154, 6)
y_train Shape: (614,)
y_test Shape: (154,)


In [5]:
#init basic CNN classifier
cnn_model = MLPClassifier(random_state=69)
cnn_model.fit(X_train, y_train)

#evaluate loss of vanilla classifier
cnn_model.loss_

0.5720681138289131

In [6]:
#generating all combinations of parameters for GridSearch
hidden_layer_sizes = [int(x) for x in np.linspace(start=80, stop=200, num=10)] # number trees in random forest
activation = ['identity', 'logistic', 'tanh', 'relu']
solver = ['lbfgs', 'sgd', 'adam']
batch_size=[50, 100, 150, 200]
learning_rate = ['constant', 'invscaling', 'adaptive']

In [7]:
# combinations --> hashtable
param_grid = {'hidden_layer_sizes': hidden_layer_sizes,
               'activation': activation,
               'solver': solver,
               'batch_size': batch_size,
               'learning_rate': learning_rate,
             }
print(param_grid)

{'hidden_layer_sizes': [80, 93, 106, 120, 133, 146, 160, 173, 186, 200], 'activation': ['identity', 'logistic', 'tanh', 'relu'], 'solver': ['lbfgs', 'sgd', 'adam'], 'batch_size': [50, 100, 150, 200], 'learning_rate': ['constant', 'invscaling', 'adaptive']}


In [8]:
# generating and fitting GridSearchCV
cnn_model = MLPClassifier()
cnn_grid = GridSearchCV(estimator = cnn_model, param_grid = param_grid, cv = 10, verbose=2, n_jobs = 4)
cnn_grid.fit(X_train, y_train)

Fitting 10 folds for each of 1440 candidates, totalling 14400 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    6.5s
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:   24.8s
[Parallel(n_jobs=4)]: Done 357 tasks      | elapsed:   56.3s
[Parallel(n_jobs=4)]: Done 640 tasks      | elapsed:  1.7min
[Parallel(n_jobs=4)]: Done 1005 tasks      | elapsed:  2.7min
[Parallel(n_jobs=4)]: Done 1450 tasks      | elapsed:  3.6min
[Parallel(n_jobs=4)]: Done 1977 tasks      | elapsed:  4.9min
[Parallel(n_jobs=4)]: Done 2584 tasks      | elapsed:  5.9min
[Parallel(n_jobs=4)]: Done 3273 tasks      | elapsed:  7.0min
[Parallel(n_jobs=4)]: Done 4042 tasks      | elapsed: 10.4min
[Parallel(n_jobs=4)]: Done 4893 tasks      | elapsed: 16.1min
[Parallel(n_jobs=4)]: Done 5824 tasks      | elapsed: 21.8min
[Parallel(n_jobs=4)]: Done 6837 tasks      | elapsed: 28.4min
[Parallel(n_jobs=4)]: Done 7930 tasks      | elapsed: 36.4min
[Parallel(n_jobs=4)]: Done 9105 tasks      | elapsed: 45.5min

GridSearchCV(cv=10, estimator=MLPClassifier(), n_jobs=4,
             param_grid={'activation': ['identity', 'logistic', 'tanh', 'relu'],
                         'batch_size': [50, 100, 150, 200],
                         'hidden_layer_sizes': [80, 93, 106, 120, 133, 146, 160,
                                                173, 186, 200],
                         'learning_rate': ['constant', 'invscaling',
                                           'adaptive'],
                         'solver': ['lbfgs', 'sgd', 'adam']},
             verbose=2)

In [9]:
# evaluating best model, fitting model
regressor = cnn_grid.best_estimator_
regressor.fit(X_train, y_train)
regressor.loss_

0.47025305310259974

In [11]:
# caching model
trained_model = 'trained_cnn_model.sav'
pickle.dump(regressor, open(trained_model, 'wb'))

In [12]:
regressor.score(X_test, y_test)

0.7402597402597403